In [1]:
# First, we import necessary libraries:
import numpy as np
import pandas as pd
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import os
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim
from torch.optim import lr_scheduler
from PIL import Image
import random

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
#create data loaders for train and test set. Need transformation of our pretrained network to do so
pretrained_weights = models.EfficientNet_B0_Weights.IMAGENET1K_V1

train_dataset = datasets.ImageFolder(root="/kaggle/input/birds23sp/birds/train", 
                                     transform=#transforms.Compose([
                                         pretrained_weights.transforms())#,
                                         #transforms.RandomHorizontalFlip(),
                                         #transforms.RandomVerticalFlip(),
                                         #transforms.RandomRotation(30),
                                         #transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1)
                                     #]))

#train_idx = []
#val_idx = []
#for i in range(len(train_dataset)):
#    if random.random()<0.9:
#        train_idx.append(i)
#    else:
#        val_idx.append(i)
        
#train_set = torch.utils.data.Subset(train_dataset, train_idx)
#val_set = torch.utils.data.Subset(train_dataset, val_idx)

train_loader = DataLoader(dataset=train_dataset,
                              batch_size=64,
                              shuffle=True,
                              pin_memory=True, num_workers=2)

#val_loader = DataLoader(dataset=val_set,
#                              batch_size=64,
#                              shuffle=True,
#                              pin_memory=True, num_workers=2)

test_dataset = datasets.ImageFolder(root="/kaggle/input/birds23sp/birds/test", transform = pretrained_weights.transforms())

test_loader = DataLoader(dataset=test_dataset,
                              batch_size=None,
                              shuffle=False,
                              pin_memory=True, num_workers=2)

class_to_idx = train_dataset.class_to_idx
idx_to_class = {int(v): int(k) for k, v in class_to_idx.items()}
print(idx_to_class[3])

100


In [3]:
#training the model
def train_model(model, criterion, optimizer, scheduler, num_epochs = 1):

    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    model.train()

    for epoch in range(num_epochs):
        print("Starting epoch: ")
        print(epoch)
        train_counter = enumerate(train_loader)
        for times, (inputs, labels) in train_counter:
            inputs = inputs.to(device)
            labels = labels.to(device)

            #zero the parameter gradients
            optimizer.zero_grad()

            #forward
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
            
            if times % 50 == 0:
                epoch_loss = running_loss / total_predictions
                epoch_accuracy = correct_predictions / total_predictions
                print("step: ", times)
                print("Loss:", epoch_loss)
                print("Accuracy:", epoch_accuracy)
            

        scheduler.step()

    return model

In [4]:
def test_model(model):
    #Denke hier iteriere ich ueber die entries ausm test set einzeln, dann kann aber auch der dataloader fuer test weg
    correct_predictions = 0
    total_predictions = 0
    
    path = []
    classes = []
    
    model.eval()
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(val_loader):
            if(i % 10 == 0):
                print("Starting iteration: ")
                print(i)
            
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            
            _, predicted = torch.max(outputs, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
        
    print("Final test accuracy = ", correct_predictions/total_predictions)
    

In [5]:
def save_model(model):
    #Denke hier iteriere ich ueber die entries ausm test set einzeln, dann kann aber auch der dataloader fuer test weg
    
    path = []
    classes = []
    
    model.eval()
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(test_loader):
            if(i % 2000 == 0):
                print("Starting iteration: ")
                print(i)
                
            inputs = inputs.unsqueeze(0)
            
            inputs = inputs.to(device)

            outputs = model(inputs)
            
            _, predictions = torch.max(outputs, 1)
            
            paths_batch = test_loader.dataset.samples[i][0]
            substring = "/kaggle/input/birds23sp/birds/test/0/"

            paths_batch = paths_batch.replace(substring, "test/")
            
            path.append(paths_batch)
            classes.append(idx_to_class[predictions[0].item()])
            
    path = np.array(path)
    classes = np.array(classes)
    inter = np.concatenate((path[:, np.newaxis], classes[:, np.newaxis]), axis=1)

    res = pd.DataFrame(data = inter, columns=['path','class'])
    
    res.to_csv('/kaggle/working/submission.csv', index=False)
        
    return res

In [6]:
def predict(net, dataloader, ofname):
    out = open(ofname, 'w')
    out.write("path,class\n")
    net.to(device)
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for i, (images, labels) in enumerate(dataloader, 0):
            if i%100 == 0:
                print(i)
            images = images.unsqueeze(0)
            images = images.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            fname, _ = dataloader.dataset.samples[i]
            out.write("test/{},{}\n".format(fname.split('/')[-1], predicted.item()))
    out.close()

In [7]:
#definition of the model
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.layers = models.efficientnet_b0(
            weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1
        )
        
        #freeze the model here so that onlz the layers of the pretrained efficient net are frozen as we don't want to retrain them
        self.freeze()

        self.layers.classifier[1] = nn.Sequential(
            nn.Linear(1280, 2048),
            nn.BatchNorm1d(2048),
            nn.Sigmoid(),
            nn.Dropout(0.07),
            nn.Linear(2048, 1024),
            nn.BatchNorm1d(1024),
            nn.Sigmoid(),
            nn.Dropout(0.07),
            nn.Linear(1024, 555),
            nn.LogSoftmax(1)
        )
    
        
    def forward(self,x):
        x = self.layers(x)
        return x

    def freeze(self):
        for param in self.layers.parameters():
            param.requires_grad = False

    def unfreeze(self):
        for param in self.layers.parameters():
            param.requires_grad = True

In [8]:
model = Model().to(device)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 148MB/s]


In [9]:
#define parameters and fucntions 
train_epochs = 8 
#loss function 
criterion = nn.NLLLoss() 
#optimizer 
optimizer1 = optim.AdamW(model.parameters(), lr=0.01) 
#LR scheduler 
lrScheduler1 = lr_scheduler.ExponentialLR(optimizer1,gamma=0.95,verbose = False) 
#Train the model 
model = train_model(model, criterion, optimizer1, lrScheduler1, num_epochs=train_epochs)

Starting epoch: 
0
step:  0
Loss: 6.32911491394043
Accuracy: 0.0
step:  50
Loss: 11.81542521832036
Accuracy: 0.00980392156862745
step:  100
Loss: 9.46081260643383
Accuracy: 0.023514851485148515
step:  150
Loss: 8.331029118291589
Accuracy: 0.04097682119205298
step:  200
Loss: 7.604453786688658
Accuracy: 0.052083333333333336
step:  250
Loss: 7.041390375312106
Accuracy: 0.06523904382470119
step:  300
Loss: 6.608407258987427
Accuracy: 0.07833264119601328
step:  350
Loss: 6.258049893582988
Accuracy: 0.09058938746438747
step:  400
Loss: 5.976915336903788
Accuracy: 0.09928304239401496
step:  450
Loss: 5.728863344488546
Accuracy: 0.11037971175166297
step:  500
Loss: 5.518772343675534
Accuracy: 0.12063373253493014
step:  550
Loss: 5.3401397784695215
Accuracy: 0.1301894283121597
step:  600
Loss: 5.190332275063741
Accuracy: 0.1374792013311148
Starting epoch: 
1
step:  0
Loss: 5.182784932691615
Accuracy: 0.13809351214208046
step:  50
Loss: 5.024809451081094
Accuracy: 0.14852005929326256
step:  100

#define parameters and fucntions
train_epochs = 1
#loss function
criterion = nn.NLLLoss()
#optimizer
optimizer1 = optim.AdamW(model.parameters(), lr=0.01)
#LR scheduler
lrScheduler1 = lr_scheduler.ExponentialLR(optimizer1,gamma=0.95,verbose = False)
#Train the model
model = train_model(model, criterion, optimizer1, lrScheduler1, num_epochs=train_epochs)

In [10]:
#test_model(model)

In [11]:
res = save_model(model)

Starting iteration: 
0
Starting iteration: 
2000
Starting iteration: 
4000
Starting iteration: 
6000
Starting iteration: 
8000
